In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import json

In [3]:
from data_engineering import de

In [4]:
df = pd.read_csv("data/movie_data.csv")

/Users/daiglechris/Git/movie_ds_model/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,13,14,15,16,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Initial analysis of dataframe
null_0 = pd.DataFrame(df.isnull().sum()).reset_index()
null_0.rename(columns={'index': 'factor', 0: 'num_null_0'}, inplace=True)
null_0.sort_values(by='num_null_0', axis=0, ascending=False, inplace=True)
null_0['miss_prop_0'] = round(null_0['num_null_0'] / df.shape[0], 4) * 100
df_info = {'obs_0':df.shape[0],
           'fac_0':df.shape[1],
           'null_0': null_0}
# Transformations to more accurately reflect data integrity issues
numeric_cols = ['budget', 'popularity', 'revenue',
                'runtime', 'vote_average', 'vote_count']
for col in numeric_cols:
    df[col] = df[col].apply(lambda x: pd.to_numeric(x, downcast='float', errors='coerce'))
df['id'] = df['id'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df['release_date'] = df['release_date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))
# Remove observations related to response variables
df = df.dropna(how='any', subset=['vote_count', 'vote_average', 'revenue'])
# Dataframe of the number of missing values after null removals
trans_0_null = pd.DataFrame(df.isnull().sum()).reset_index()
trans_0_null.rename(columns={'index': 'factor', 0: 'num_null_1'}, inplace=True)
trans_0_null['miss_prop_1'] = round(trans_0_null['num_null_1'] / df.shape[0], 4) * 100
null_1 = null_0.join(trans_0_null.set_index('factor'), on='factor')
# Update dataframe information dictionary
df_info['obs_1'] = df.shape[0]
df_info['fac_1'] = df.shape[1]
df_info['null_1'] = null_1

In [6]:
print("Initial observations: {:,}".format(df_info['obs_0']))
print("Number of observations after removing missing 'vote_count', 'vote_average', 'revenue': {:,}".format(df_info['obs_1']))

Initial observations: 133,744
Number of observations after removing missing 'vote_count', 'vote_average', 'revenue': 28,477


In [7]:
df_info['null_1'].sort_values('miss_prop_1', ascending=False)

,factor,num_null_0,miss_prop_0,num_null_1,miss_prop_1
2,homepage,114113,85.32,24461,85.90
15,tagline,120766,90.30,15499,54.43
6,overview,33226,24.84,458,1.61
12,runtime,105434,78.83,170,0.60
10,release_date,105375,78.79,111,0.39
13,spoken_languages,105264,78.71,0,0.00
3,id,32768,24.50,0,0.00
4,original_language,32768,24.50,0,0.00
5,original_title,32768,24.50,0,0.00
1,genres,32768,24.50,0,0.00


In [8]:
for col in df.columns:
    print(col)
    print(df[col].dtypes)
    print("{:.2%} null".format(df[col].isna().sum() / df.shape[0]))
    display(df[col].head())
    print("+" * len(col))

budget
float64
0.00% null


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: budget, dtype: float64

++++++
genres
object
0.00% null


0                       [{"id": 35, "name": "Comedy"}]
1                    [{"id": 12, "name": "Adventure"}]
2    [{"id": 10749, "name": "Romance"}, {"id": 18, ...
3                       [{"id": 35, "name": "Comedy"}]
4                                                   []
Name: genres, dtype: object

++++++
homepage
object
85.90% null


...

++++++++++++
vote_count
float64
0.00% null


0     4.0
1     0.0
2    27.0
3     0.0
4     2.0
Name: vote_count, dtype: float64

++++++++++
keywords
object
0.00% null


0                  [{"id": 190801, "name": "blondie"}]
1                                                   []
2                                                   []
3                                                   []
4    [{"id": 6075, "name": "sport"}, {"id": 10192, ...
Name: keywords, dtype: object

++++++++


## Data Engineering Strategy
For each column in the dataframe, here's some info and my plan to handle them:
1. **budget**: float64, 0% null
> *ALREADY CONVERTED TO FLOAT AND COERCED NAN*
2. **genres**: object, 0% null, array holding dictionary, ex: `x = [{"id": 35, "name": "Comedy"}]`
> *DRAW VALUE FROM "name" FROM EACH ELEMENT FROM ARRAY, ASSIGN NAN IF EMPTY*
3. **homepage**: object, 85.9% null
> *DROP: TOO MUCH MISSING*
4. **id**: float64, 0% null
> *DROP: NO INFORMATION*
5. **original_language**: object, 0% null
> *ENFORCE STRING*
6. **original_title**: object, 0% null
> *ENFORCE STRING*
7. **overview**: object, 1.61% null
> *ENFORCE STRING*
8. **popularity**: float64, 0% null
9. **production_companies**: object, 0% null, array holding dictionary
> *DRAW VALUE FROM "name" FROM EACH ELEMENT FROM ARRAY, ASSIGN NAN IF EMPTY*
10. **production_countries**: object, 0% null, array holding dictionaries
> *DRAW VALUE FROM "iso_3166_1" FROM EACH ELEMENT FROM ARRAY, ASSIGN NAN IF EMPTY*
11. **release_date**: datetime64, 0.39% null
> *ALREADY CONVERTED TO DATETIME AND FILLED NAN*
12. **revenue**: float64, 0% null
> *RESPONSE VARIABLE*
13. **runtime**: float64, 0.6% null
> *ALREADY CONVERTED TO FLOAT AND COERCED NAN*
14. **spoken_languages**: object, 0% null, array of dictionaries
> *Drop and see if valuable after using only original_language*<br>
> If brought back in...<br>
> *DRAW VALUE FROM "iso_639_1" FROM EACH ELEMENT FROM ARRAY, ASSIGN NAN IF EMPTY, CAPITALIZE*
15. **status**: object, 0% null
> *CHECK IF BINARY INDICATOR*
16. **tagline**: object, 54.43% null
> *FIND RELATIONSHIP BETWEEN ORIGINAL_LANGUAGE, PRODUCTION_COUNTRIES, AND STATUS*
17. **title**: object, 0% null
18. **vote_average**: float64, 0% null
> *PREVIOUSLY CONVERTED TO FLOAT AND DROPPED MISSING*
19. **vote_count**: float64, 0% null
> *PREVIOUSLY CONVERTED TO FLOAT AND DROPPED MISSING*
20. **keywords**: object, 0% null, array holding dictionaries
> *DRAW VALUE FROM "name" FROM EACH ELEMENT FROM ARRAY, ASSIGN NAN IF EMPTY*

In [9]:
df['genres'] = de.string_array_clean(df_col=df['genres'], element='name')
df.drop(['homepage', 'id'], axis=1, inplace=True)
df['original_language'] = df['original_language'].astype(str)
df['original_title'] = df['original_title'].astype(str)
df['overview'] = df['overview'].astype(str)
df['production_companies'] = de.string_array_clean(df_col=df['production_companies'], element='name')
df['production_countries'] = de.string_array_clean(df_col=df['production_countries'], element='iso_3166_1')
df['spoken_languages'] = de.string_array_clean(df_col=df['spoken_languages'], element='iso_639_1')
df['tagline'] = df['tagline'].astype(str)
df['title'] = df['title'].astype(str)
df['keywords'] = de.string_array_clean(df_col=df['keywords'], element='name')
df.reset_index(inplace = True, drop = True)

In [10]:
df.head()

,budget,genres,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,keywords
0,0.0,[Comedy],en,Blondie,Blondie and Dagwood are about to celebrate the...,3.067,[Columbia Pictures],[US],1938-11-30,0.0,70.0,[en],Released,nan,Blondie,6.6,4.0,[blondie]
1,0.0,[Adventure],de,Der Mann ohne Namen,Der Mann ohne Namen is a German adventure movi...,0.845,NaN,[DE],1921-01-01,0.0,420.0,NaN,Released,nan,"Peter Voss, Thief of Millions",0.0,0.0,NaN
2,0.0,"[Romance, Drama]",fr,L'amour à vingt ans,Love at Twenty unites five directors from five...,3.701,"[Ulysse Productions, Unitec Films, Cinesecolo,...","[DE, FR, IT, JP, PL]",1962-06-22,0.0,110.0,"[de, fr, it, ja, pl]",Released,The Intimate Secrets of Young Lovers,Love at Twenty,6.9,27.0,NaN
3,0.0,[Comedy],pa,Jatts in Golmaal,Jatts in Golmal is an Comedy based movie. In w...,0.600,NaN,NaN,2003-02-21,0.0,NaN,NaN,Released,nan,Jatts in Golmaal,0.0,0.0,NaN
4,0.0,NaN,en,New World Disorder 9: Never Enough,Gee Atherton ripping the Worlds course the day...,1.212,NaN,NaN,2008-12-08,0.0,69.0,[en],Released,nan,New World Disorder 9: Never Enough,4.5,2.0,"[sport, mountain bike]"


In [11]:
# Initial analysis of dataframe
trans_null_2 = pd.DataFrame(df.isnull().sum()).reset_index()
trans_null_2.rename(columns={'index': 'factor', 0: 'num_null_2'}, inplace=True)
null_2 = null_1.join(trans_null_2.set_index('factor'), on='factor')
df_info['obs_2'] = df.shape[0]
df_info['fac_2'] = df.shape[1]
df_info['null_2'] = null_2

In [12]:
df_info['null_2']['miss_prop_2'] = round(df_info['null_2']['num_null_2'] / df.shape[0], 4) * 100
df_info['null_2'].sort_values('miss_prop_2', ascending=False)

,factor,num_null_0,miss_prop_0,num_null_1,miss_prop_1,num_null_2,miss_prop_2
19,keywords,105267,78.71,0,0.00,7276.0,25.55
8,production_companies,105264,78.71,0,0.00,6877.0,24.15
9,production_countries,105267,78.71,0,0.00,2849.0,10.00
13,spoken_languages,105264,78.71,0,0.00,1754.0,6.16
1,genres,32768,24.50,0,0.00,1217.0,4.27
12,runtime,105434,78.83,170,0.60,170.0,0.60
10,release_date,105375,78.79,111,0.39,111.0,0.39
15,tagline,120766,90.30,15499,54.43,0.0,0.00
7,popularity,105264,78.71,0,0.00,0.0,0.00
4,original_language,32768,24.50,0,0.00,0.0,0.00


In [17]:
unique_kw = de.uniques_from_list(df['keywords'])

True

In [33]:
df['keywords'].shape[0]

28477

In [38]:
df['keywords'].tail()

133739                              [inheritance challenge]
133740    [world war ii, parachuting, war correspondent,...
133741    [composer, musical, biography, broadway, gersh...
133742    [wales, based on play or musical, teacher, stu...
133743                                   [independent film]
Name: keywords, dtype: object

In [39]:
df['keywords'][133743]

['independent film']